In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv3D
from keras.layers import AveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization,ReLU
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.models import Sequential


In [ ]:
input_layer = Input(shape=(299, 299, 3))
model = Create_inception_model(input_layer,1000)

plot_model(model, show_shapes=True, to_file='Vgg.png')



In [ ]:
def Create_inception_model(input_layer,number_of_classes):


  stem_layer = Calculate_stem(input_layer)

  out_A_1 = ModuleA_block(stem_layer,64,48,64,64,96,32)
  out_A_2 = ModuleA_block(out_A_1,64,48,64,64,96,64)
  out_A_3 = ModuleA_block(out_A_2,64,48,64,64,96,64)
  grid = grid_size_reduction1(out_A_3,384,64,96)
  
  out_B_1 = ModuleB_block(grid,192,128,192,128,192,192)
  out_B_2 = ModuleB_block(out_B_1,192,160,192,160,192,192)
  out_B_3 = ModuleB_block(out_B_2,192,160,192,160,192,192)
  out_B_4 = ModuleB_block(out_B_3,192,192,192,192,192,192)
  out_B_5 = ModuleB_block(out_B_4,192,128,192,128,192,192)

  grid2 = grid_size_reduction2(out_B_5,192,320,192)

  out_C_1 = ModuleC_block(grid2)
  out_C_2 = ModuleC_block(out_C_1)
 

  output = GlobalAveragePooling2D()(out_C_2)
  output = Dense(number_of_classes, activation='softmax', name='output')(output)

  model = Model(input_layer, output)
  return model


In [ ]:
def Calculate_stem(input_layer): 
  
  
  conv1 = Conv2D(32, (3, 3), padding='same', strides=(2, 2))(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)
  
  conv2 = Conv2D(32, (3, 3), padding='same', strides=(1, 1))(relu_conv1)
  norm_conv2 = BatchNormalization()(conv2)
  relu_conv2 = ReLU()(norm_conv2)

  conv3 = Conv2D(64, (3, 3), padding='same', strides=(1, 1))(relu_conv2)
  norm_conv3 = BatchNormalization()(conv3)
  relu_conv3 = ReLU()(norm_conv3)

  pool = MaxPooling2D((3,3), strides=(2,2), padding='same')(relu_conv3)

 
  conv4 = Conv2D(80, (1, 1), padding='same', strides=(1, 1))(pool)
  norm_conv4 = BatchNormalization()(conv4)
  relu_conv4 = ReLU()(norm_conv4)

  conv5 = Conv2D(192, (3, 3), padding='same', strides=(1, 1))(relu_conv4)
  norm_conv5 = BatchNormalization()(conv5)
  relu_conv5 = ReLU()(norm_conv5)
  
  out =  MaxPooling2D()(relu_conv5)
  # conv6 = Conv2D(256, (3, 3), padding='same', strides=(1, 1))(relu_conv5)
  # norm_conv6 = BatchNormalization()(conv6)
  # relu_conv6 = ReLU()(norm_conv6)

  return  out

In [ ]:
def ModuleA_block(input_layer,filter1_1,filter2_1,filter2_2,filter3_1,filter3_2,filter4_1):
  branch1 = ModuleA_Branch_1(input_layer,filter1_1)
  branch2 = ModuleA_Branch_3(input_layer,filter2_1,filter2_2)
  branch3 = ModuleA_Branch_5(input_layer,filter3_1,filter3_2)
  pool_Branch = ModuleA_Pool(input_layer,filter4_1)

  output_layer  = concatenate([branch1,branch2, branch3, pool_Branch], axis=-1)
  return output_layer

def ModuleB_block(input_layer,filter1_1,filter2_1,filter2_2,filter3_1,filter3_2,filter4_1):
  branch1 = ModuleB_Branch_1(input_layer,filter1_1)#192
  branch2 = ModuleB_Branch_7(input_layer,filter2_1,filter2_2)#128,192,256
  branch3 = ModuleB_Branch_2x7(input_layer,filter3_1,filter3_2)#128,192,256
  pool_Branch = ModuleB_Pool(input_layer,filter4_1)#192

  output_layer  = concatenate([branch1,branch2, branch3, pool_Branch], axis=-1)
  return output_layer

def ModuleC_block(input_layer):
  branch1 = ModuleC_Branch_1(input_layer,320)
  branch2 = ModuleC_Branch_3(input_layer,384) 
  branch3 = ModuleC_Branch_5(input_layer,448,384)

  pool_Branch = ModuleC_Pool(input_layer,192)
  output_layer  = concatenate([branch1,branch2, branch3, pool_Branch], axis=-1)
  return output_layer

In [ ]:
#Helper functions that will help in creating the inception layers = Module A

def ModuleA_Branch_1(input_layer,filter1):
  
  conv = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm = BatchNormalization()(conv)
  relu = ReLU()(norm)
  return relu

def ModuleA_Branch_3(input_layer,filter1,filter3):

  conv1 = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)

  conv3_1 = Conv2D(filter3, (3,3), padding='same')(relu_conv1)
  norm_conv3_1 = BatchNormalization()(conv3_1)
  relu_conv3_1 = ReLU()(norm_conv3_1) 
  return relu_conv3_1

def ModuleA_Branch_5(input_layer,filter1,filter3):
  
  conv1 = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)

  conv3_1 =  Conv2D(filter3, (3,3), padding='same')(relu_conv1)
  norm_conv3_1 = BatchNormalization()(conv3_1)
  relu_conv3_1 = ReLU()(norm_conv3_1)

  conv3_3 =  Conv2D(filter3, (3,3), padding='same')(relu_conv3_1)
  norm_conv3_3 = BatchNormalization()(conv3_3)
  relu_conv3_3 = ReLU()(norm_conv3_3)
  return relu_conv3_3

def ModuleA_Pool(input_layer,filter):
  pool = AveragePooling2D((3,3), strides=(1,1), padding='same')(input_layer)  
  
  conv1_pool = Conv2D(filter, (1,1), padding='same')(pool)
  norm_conv1_pool = BatchNormalization()(conv1_pool)
  relu_conv1_pool = ReLU()(norm_conv1_pool)
  return relu_conv1_pool

In [ ]:
#Helper functions that will help in creating the inception layers = Module B
def ModuleB_Branch_1(input_layer,filter):
  conv = Conv2D(filter, (1,1), padding='same')(input_layer)
  norm = BatchNormalization()(conv)
  relu = ReLU()(norm)
  return relu

def ModuleB_Branch_7(input_layer,filter1,filter2):
  
  conv1 = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)

  conv_1x7_1 = Conv2D(filter1, (1,7), padding='same')(relu_conv1)
  norm_conv_1x7_1 = BatchNormalization()(conv_1x7_1)
  relu_conv_1x7_1 = ReLU()(norm_conv_1x7_1)

  conv_7x1_1x7 = Conv2D(filter2, (7,1), padding='same')(relu_conv_1x7_1)
  norm_conv_7x1_1x7 = BatchNormalization()(conv_7x1_1x7)
  relu_conv_7x1_1x7 = ReLU()(norm_conv_7x1_1x7) 

  return relu_conv_7x1_1x7

def ModuleB_Branch_2x7(input_layer,filter1,filter2):

  conv1 = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)

  conv_1x7_1 = Conv2D(filter1, (1,7), padding='same')(relu_conv1)
  norm_conv_1x7_1 = BatchNormalization()(conv_1x7_1)
  relu_conv_1x7_1 = ReLU()(norm_conv_1x7_1)

  conv_7x1_1x7 = Conv2D(filter1, (7,1), padding='same')(relu_conv_1x7_1) 
  norm_conv_7x1_1x7 = BatchNormalization()(conv_7x1_1x7)
  relu_conv_7x1_1x7 = ReLU()(norm_conv_7x1_1x7)

  conv_1x7_7x1 = Conv2D(filter1, (1,7), padding='same')(relu_conv_7x1_1x7)
  norm_conv_1x7_7x1 = BatchNormalization()(conv_1x7_7x1)
  relu_conv_1x7_7x1 = ReLU()(norm_conv_1x7_7x1)

  conv_7x1_1x7_new = Conv2D(filter2, (7,1), padding='same')(relu_conv_1x7_7x1)
  norm_conv_7x1_1x7_new = BatchNormalization()(conv_7x1_1x7_new)
  relu_conv_7x1_1x7_new = ReLU()(norm_conv_7x1_1x7_new) 
  
  return relu_conv_7x1_1x7_new

def ModuleB_Pool(input_layer,filter):
  pool = AveragePooling2D((3,3), strides=(1,1), padding='same')(input_layer)  
  conv1_pool = Conv2D(filter, (1,1), padding='same')(pool)
  norm_conv1_pool = BatchNormalization()(conv1_pool)
  relu_conv1_pool = ReLU()(norm_conv1_pool)
  return relu_conv1_pool

In [ ]:
#Helper functions that will help in creating the inception layers = Module C
def ModuleC_Branch_1(input_layer,filter):
  conv = Conv2D(filter, (1,1), padding='same')(input_layer)
  norm = BatchNormalization()(conv)
  relu = ReLU()(norm)
  return relu

def ModuleC_Branch_3(input_layer,filter1):

  conv1 = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)

  conv_3x1_1 = Conv2D(filter1, (3,1), padding='same')(relu_conv1)
  norm_conv_3x1_1 = BatchNormalization()(conv_3x1_1)
  relu_conv_3x1_1 = ReLU()(norm_conv_3x1_1)


  conv_1x3_1 = Conv2D(filter1, (1,3), padding='same')(relu_conv1)
  norm_conv_1x3_1 = BatchNormalization()(conv_1x3_1)
  relu_conv_1x3_1 = ReLU()(norm_conv_1x3_1)
  

  conca_block = concatenate([relu_conv_3x1_1,relu_conv_1x3_1], axis=-1)

  return conca_block

def ModuleC_Branch_5(input_layer,filter1,filter2):

  conv1 = Conv2D(filter1, (1,1), padding='same')(input_layer)
  norm_conv1 = BatchNormalization()(conv1)
  relu_conv1 = ReLU()(norm_conv1)

  conv_1x3_1 = Conv2D(filter2, (1,3), padding='same')(relu_conv1)
  norm_conv_1x3_1 = BatchNormalization()(conv_1x3_1)
  relu_conv_1x3_1 = ReLU()(norm_conv_1x3_1)

  conv_3x1_1x3 = Conv2D(filter2, (3,1), padding='same')(relu_conv_1x3_1)
  norm_conv_3x1_1x3 = BatchNormalization()(conv_3x1_1x3)
  relu_conv_3x1_1x3 = ReLU()(norm_conv_3x1_1x3)

  conv_3x1_par = Conv2D(filter2, (3,1), padding='same')(relu_conv_3x1_1x3)
  norm_conv_3x1_par = BatchNormalization()(conv_3x1_par)
  relu_conv_3x1_par = ReLU()(norm_conv_3x1_par)
  
  conv_1x3_par = Conv2D(filter2, (1,3), padding='same')(relu_conv_3x1_1x3)
  norm_conv_1x3_par = BatchNormalization()(conv_1x3_par)
  relu_conv_1x3_par = ReLU()(norm_conv_1x3_par)


  conca_block = concatenate([relu_conv_3x1_par,relu_conv_1x3_par], axis=-1)

  return conca_block

def ModuleC_Pool(input_layer,filter):
    pool = AveragePooling2D((3,3), strides=(1,1), padding='same')(input_layer) 

    conv1_pool = Conv2D(filter, (1,1), padding='same')(pool)
    norm_conv1_pool = BatchNormalization()(conv1_pool)
    relu_conv1_pool = ReLU()(norm_conv1_pool)
    return relu_conv1_pool



In [ ]:
def grid_size_reduction1(input_layer,filter1,filter3_1,filter3_2):

  pool = MaxPooling2D((1,1) , strides=2,padding='same')(input_layer)

  conv_1 = Conv2D(filter1, (1,1),strides=2 ,padding='same')(input_layer)
  norm_conv_1 = BatchNormalization()(conv_1)
  relu_conv_1 = ReLU()(norm_conv_1)

  conv_2_1 = Conv2D(filter3_1, (1,1) ,strides=1 ,padding='same')(input_layer)
  norm_conv_2_1 = BatchNormalization()(conv_2_1)
  relu_conv_2_1 = ReLU()(norm_conv_2_1)

  conv_2_2 = Conv2D(filter3_2, (3,3),strides=1 ,padding='same')(relu_conv_2_1)
  norm_conv_2_2 = BatchNormalization()(conv_2_2)
  relu_conv_2_2 = ReLU()(norm_conv_2_2)

  conv_2_3 = Conv2D(filter3_2, (3,3),strides=2 ,padding='same')(relu_conv_2_2)
  norm_conv_2_3 = BatchNormalization()(conv_2_3)
  relu_conv_2_3 = ReLU()(norm_conv_2_3)

  output = concatenate([relu_conv_2_3, relu_conv_1, pool], axis=-1)
  return output

In [ ]:
def grid_size_reduction2(input_layer,filter1_1,filter1_2,filter2):

  pool = MaxPooling2D((1,1) , strides=2,padding='same')(input_layer)

  conv_1 = Conv2D(filter1_1, (1,1),strides=1 ,padding='same')(input_layer)
  norm_conv_1 = BatchNormalization()(conv_1)
  relu_conv_1 = ReLU()(norm_conv_1)

  conv_1_2 = Conv2D(filter1_2, (1,1),strides = 2 ,padding='same')(relu_conv_1)
  norm_conv_1_2 = BatchNormalization()(conv_1_2)
  relu_conv_1_2 = ReLU()(norm_conv_1_2)

  conv_2_1 = Conv2D(filter2, (1,1) ,strides=1 ,padding='same')(input_layer)
  norm_conv_2_1 = BatchNormalization()(conv_2_1)
  relu_conv_2_1 = ReLU()(norm_conv_2_1)

  conv_2_2 = Conv2D(filter2, (3,3),strides=1 ,padding='same')(relu_conv_2_1)
  norm_conv_2_2 = BatchNormalization()(conv_2_2)
  relu_conv_2_2 = ReLU()(norm_conv_2_2)

  conv_2_3 = Conv2D(filter2, (3,3),strides=1 ,padding='same')(relu_conv_2_2)
  norm_conv_2_3 = BatchNormalization()(conv_2_3)
  relu_conv_2_3 = ReLU()(norm_conv_2_3)

  
  conv_2_4 = Conv2D(filter2, (3,3),strides=2 ,padding='same')(relu_conv_2_3)
  norm_conv_2_4 = BatchNormalization()(conv_2_4)
  relu_conv_2_4 = ReLU()(norm_conv_2_4)

  output = concatenate([relu_conv_2_4, relu_conv_1_2, pool], axis=-1)
  return output